In [ ]:
from tensorflow import keras
from keras.models import Model
from keras import layers
from keras.layers import Dense, Input, BatchNormalization, Activation
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

from keras.datasets import mnist
from PIL import Image
# for creating a one hot vector for labels
from keras.utils import np_utils
from IPython.display import display, Image
#import the models
from keras import Model
#add layers
from keras import layers
#add optimizer
from keras import optimizers
#add loss function 
from keras import losses

import keras
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
 

Source : https://github.com/RayXie29/Keras-famous_CNN/blob/master/InceptionV1.py

In [ ]:
"""
Skip to content
Search or jump to…

Pull requests
Issues
Marketplace
Explore
 
@ragib1996 
RayXie29
/
Keras-famous_CNN
1
84
Code
Issues
Pull requests
Actions
Projects
Wiki
Security
Insights
Keras-famous_CNN/InceptionV1.py /
@RayXie29
RayXie29 add inception block name
Latest commit f13c804 on Aug 11, 2019
 History
 1 contributor
181 lines (132 sloc)  8.03 KB
"""
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation
from keras.layers import Flatten, Input, Dropout, concatenate
from keras.regularizers import  l2
from keras import backend as K
from keras.models import Model

img_size = 299

class Inceptionv1_builder():

    def __init__(self, input_shape = (img_size, img_size ,3), output_units = 3, init_kernel = (7,7), init_strides = (2,2), init_filters = 64,
                 regularizer = l2(1e-4), initializer = "he_normal", init_maxpooling = True):
        '''
        :param input_shape: input shape of dataset
        :param output_units: output result dimension
        :param init_kernel: The kernel size for first convolution layer
        :param init_strides: The strides for first convolution layer
        :param init_filters: The filter number for first convolution layer
        :param regularizer: regularizer for all the convolution layers in whole NN
        :param initializer: weight/parameters initializer for all convolution & fc layers in whole NN
        :param init_maxpooling: Do the maxpooling after first two convolution layers or not
        '''

        assert len(input_shape) == 3, "input shape should be dim 3 ( row, col, channel or channel row col )"

        self.input_shape = input_shape
        self.output_units = output_units
        self.init_kernel = init_kernel
        self.init_strides = init_strides
        self.init_filters = init_filters
        self.regularizer = regularizer
        self.initializer = initializer
        self.init_maxpooling = init_maxpooling

        #if K.image_dim_ordering() == "tf":

        self.row_axis = 1
        self.col_axis = 2
        self.channel_axis = 3

        """
        else:

            self.row_axis = 2
            self.col_axis = 3
            self.channel_axis = 1
        """


    def _cn_relu(self, filters = 64, kernel_size = (3,3), strides = (1,1), padding = "same"):
        '''
        convenient function to build convolution(with regularizer and initializer) -> relu activation layers
        '''
        def f(input_x):

            x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding,activation="relu",
                       kernel_initializer = self.initializer , kernel_regularizer = self.regularizer)(input_x)
            return x

        return f

    def _auxiliary(self, name = "auxiliary_1"):
        '''
        In author's explanation:
        " The auxiliary classifier will encourage discrimination in lower stages in the classifier,
        increase the gradient signal that gets propagated back, and provide additional regularization"
        :return: An output layer of auxiliary classifier
        '''
        def f(input_x):

            x = input_x
            x = AveragePooling2D(pool_size=(5,5), strides = (3,3), padding = "same")(x)
            x = self._cn_relu(filters = 128, kernel_size = (1,1), padding = "same")(x)
            x = Flatten()(x)
            x = Dense(units = 1024, activation = "relu", kernel_regularizer= self.regularizer)(x)
            x = Dropout(0.7)(x)

            return Dense(units = self.output_units, activation = "softmax", kernel_initializer=self.initializer, name = name)(x)

        return f


    def _inception_block(self, _1x1 = 64, _3x3r = 96, _3x3 = 128, _5x5r = 16, _5x5 = 32, _maxpool = 32, name = "inception3a"):
        '''
        A function for building inception block, including 1x1 convolution layer, 3x3 convolution layer with dimension reducing,
        5x5 convolution layer with dimension reducing and maxpooling layer with dimension reducing
        :param _1x1: filter number of 1x1 convolution layer
        :param _3x3r: filter number of dimension reducing layer for 3x3 convolution layer
        :param _3x3: filter number of 3x3 convolution layer
        :param _5x5r: filter number of dimension reducing layer for 5x5 convolution layer
        :param _5x5: filter number of 5x5 convolution layer
        :param _maxpool: filter number of dimension reducing layer for maxpooling layer
        :return: A concatenate block of several scale convolution which is inception block
        '''
        def f(input_x):

            branch1x1 = self._cn_relu(filters=_1x1, kernel_size=(1, 1), strides=(1, 1), padding="same")(input_x)

            branch3x3 = self._cn_relu(filters=_3x3r, kernel_size=(1, 1), strides=(1, 1), padding="same")(input_x)
            branch3x3 = self._cn_relu(filters=_3x3, kernel_size=(3, 3), strides=(1, 1), padding="same")(branch3x3)

            branch5x5 = self._cn_relu(filters=_5x5r, kernel_size=(1, 1), strides=(1, 1), padding="same")(input_x)
            branch5x5 = self._cn_relu(filters=_5x5, kernel_size=(5, 5), strides=(1, 1), padding="same",)(branch5x5)

            brancemaxpool = MaxPooling2D(pool_size = (3,3), strides = (1,1), padding = "same")(input_x)
            brancemaxpool = self._cn_relu(filters=_maxpool, kernel_size=(1, 1), strides=(1, 1), padding="same")(brancemaxpool)

            return concatenate([branch1x1,branch3x3,branch5x5,brancemaxpool], axis = self.channel_axis, name = name)

        return f
    def build_inception(self):

        '''
        Main function for building inceptionV1 nn
        :return: An inceptionV1 nn
        '''

        #Few traditional convolutional layers at lower layers
        input_x = Input(self.input_shape)

        x = self._cn_relu(filters = self.init_filters, kernel_size = self.init_kernel, strides = self.init_strides, padding = "same")(input_x)

        if self.init_maxpooling:
            x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = "same")(x)

        x = self._cn_relu(filters = 192, kernel_size = (3,3), strides = (1, 1), padding = "same")(x)

        if self.init_maxpooling:
            x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = "same")(x)

        #inception(3a)
        x = self._inception_block(_1x1=64, _3x3r=96, _3x3=128, _5x5r=16, _5x5=32, _maxpool=32, name = "inception3a")(x)

        #inception(3b)
        x = self._inception_block(_1x1=128, _3x3r=128, _3x3=192, _5x5r=32, _5x5=96, _maxpool=64, name = "inception3b")(x)

        x = MaxPooling2D(pool_size=(3,3), strides = (2,2), padding = "same")(x)

        #inception(4a)
        x = self._inception_block(_1x1=192, _3x3r=96, _3x3=208, _5x5r=16, _5x5=48, _maxpool=64, name = "inception4a")(x)

        #auxiliary classifier 1
        auxiliary1 = self._auxiliary(name = "auxiliary_1")(x)

        # inception(4b)
        x = self._inception_block(_1x1=160, _3x3r=112, _3x3=224, _5x5r=24, _5x5=64, _maxpool=64, name = "inception4b")(x)
        # inception(4c)
        x = self._inception_block(_1x1=128, _3x3r=128, _3x3=256, _5x5r=24, _5x5=64, _maxpool=64, name = "inception4c")(x)
        # inception(4d)
        x = self._inception_block(_1x1=112, _3x3r=144, _3x3=288, _5x5r=32, _5x5=64, _maxpool=64, name = "inception4d")(x)

        #auxiliary classifier 2
        auxiliary2 = self._auxiliary(name = "auxiliary_2")(x)

        # inception(4e)
        x = self._inception_block(_1x1=256, _3x3r=160, _3x3=320, _5x5r=32, _5x5=128, _maxpool=128, name = "inception4e")(x)

        x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding = "same")(x)

        #inception(5a)
        x = self._inception_block(_1x1=256, _3x3r=160, _3x3=320, _5x5r=32, _5x5=128, _maxpool=128, name = "inception5a")(x)
        #inception(5b)
        x = self._inception_block(_1x1=384, _3x3r=192, _3x3=384, _5x5r=48, _5x5=128, _maxpool=128, name = "inception5b")(x)

        x_shape = K.int_shape(x)
        x = AveragePooling2D(pool_size = (x_shape[self.row_axis], x_shape[self.col_axis]), strides=(1,1))(x)
        x = Flatten()(x)
        x = Dropout(0.4)(x)
        x = Dense(units = 1000, kernel_initializer = self.initializer, activation="relu")(x)
        output_x = Dense(units = self.output_units, activation = "softmax", kernel_initializer=self.initializer, name = "main_output")(x)

        inceptionv1_model = Model(inputs = [input_x], outputs = [auxiliary1, auxiliary2, output_x])

        return inceptionv1_model


inception_builder = Inceptionv1_builder()
model = inception_builder.build_inception()
model.compile(
    optimizer= 'Adam',
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)
model.summary()



In [ ]:
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []

images = []

#valid_covid = "../input/radiography-dataset/new_Dataset/test/Covid"
covid = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID"
covid_list = os.listdir(covid)

pneumonia = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia"
pneumonia_list = os.listdir(pneumonia)

normal = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal"
normal_list = os.listdir(normal)


csv_file_link = "../input/radiography-database-springer-2-image-division/division_into_train_valid__test.csv"
csv_file = pd.read_csv(csv_file_link)
print(csv_file.columns)


group_by = csv_file.groupby('image_type')
for i,j in group_by:
    print(i)
    
    usedin_group_by = j.groupby('used_in')
    for k,l in usedin_group_by:
        print(k)
        print(len(l))
        for m in range(0, len(l)):
            if i == 'covid':
                    
                    link = covid + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
                    if k == 'validation':
                        x_validation.append(image)
                        label_validation.append([0])
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([0])
                    if k == 'train':
                        x.append(image)
                        label.append([0])

                      
            if i == 'pneumonia':
                    
                    link = pneumonia + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
                    if k == 'validation':
                        x_validation.append(image)
                        label_validation.append([1])
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([1])
                    if k == 'train':
                        x.append(image)
                        label.append([1])
            
          
         
                        
                        
            if i == 'normal':
                    
                    link = normal + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
                    if k == 'validation':
                        x_validation.append(image)
                        label_validation.append([2])
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([2])
                    if k == 'train':
                        x.append(image)
                        label.append([2])
                    
 
                        
x = np.asarray(x)
print(len(x))


x_validation = np.asarray(x_validation)
print(len(x_validation))




x_test = np.asarray(x_test)
print(len(x_test))



#label = np.array(label)
label = keras.utils.np_utils.to_categorical(label , 3 )
#print(label)
print(len(label))


#label = np.array(label)
label_validation = keras.utils.np_utils.to_categorical(label_validation , 3 )
#print(label_validation)
print(len(label_validation))



#label_validation = np.array(label_validation)
label_test = keras.utils.np_utils.to_categorical(label_test , 3 )
#print(label_test)
print(len(label_test ))





In [ ]:
early_stop  = tf.keras.callbacks.EarlyStopping(monitor='val_main_output_acc', patience= 5, verbose=0, mode='max')
check_point = tf.keras.callbacks.ModelCheckpoint('./Inception_V1_Springer_paper_2.h5', monitor='val_main_output_acc', save_best_only=True, mode='max')
history = model.fit(x , label , validation_data = (x_validation, label_validation), epochs = 7, callbacks=[early_stop, check_point])
    

print("skip")

In [ ]:
import pickle

with open('./Inception_V1_Springer_paper_2.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

print("pickle_dumped")

In [ ]:
# convert the history.history dict to a pandas DataFrame: 
import pandas as pd
hist_df = pd.DataFrame(history.history) 

hist_csv_file = './Inception_V1_Springer_2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
print("skip")

In [ ]:
extract_saved_model = tf.keras.models.load_model("./Inception_V1_Springer_paper_2.h5")
#extract_saved_model.summary()
score = extract_saved_model.evaluate(x_test, label_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * score[6]))

In [ ]:
acc = history.history['main_output_acc']
val_acc = history.history['val_main_output_acc']

loss = history.history['main_output_loss']
val_loss = history.history['val_main_output_loss']

epochs = 7
epochs_range = range(epochs)

plt.figure(figsize=(8, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.savefig("Inception_V2_Accuracy.png")
plt.show()


In [ ]:
plt.figure(figsize=(8 , 5))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig("Inception_V2_loss.png")
plt.show()

In [ ]:
while(True):
    i = 0 
    
    
    
    